In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
pip install -r /content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 875 kB 6.5 MB/s 
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3-py3-none-any.whl size=2983 sha256=7aed59d409e485e6577a043b4c284c6c9cb96de01a6daa9362a2025d7cf5d622
  Stored in directory: /root/.cache/pip/wheels/f2/ce/4a/215c4f0add432420ff90fe04656bf2664ddfac7302e2b6fe51
Successfully built warmup-scheduler


In [3]:
import sys
#sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')
sys.path.append('/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet')

### Thư viện cần thiết

In [4]:
from tslearn.metrics import dtw, dtw_path
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from warmup_scheduler import GradualWarmupScheduler
from utils.cyclic_scheduler import CyclicLRWithRestarts
from utils.dilate_loss import dilate_loss
from utils.adamw import AdamW
from utils.metrics import RMSLE
from utils.support import *
from utils.prepare_QLD import test_qld_single_station
from utils.prepare_QLD_Level import test_qld_single_station_Level
from utils.early_stopping import EarlyStopping
from models.DualHead_NoShare import Shared_Encoder, Cross_Attention, Decoder, DualSSIM
import pandas as pd
import torch, random, math, os, time 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR, ExponentialLR

import numpy as np
np.set_printoptions(threshold=np.inf)


SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Các hàm cần thiết để train mô hình

In [5]:
# Các siêu tham số của mô hình
INPUT_DIM = 6
OUTPUT_DIM = 1
ENC_HID_DIM = 50
DEC_HID_DIM = 50
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
ECN_Layers = 1
DEC_Layers = 1
LR = 0.001  # learning rate
CLIP = 1
EPOCHS = 50 ########
BATCH_SIZE = 1
N_output = 6

In [6]:
def train(model, optimizer, criterion, X_train_left, X_train_right, y_train):

    iter_per_epoch = int(np.ceil(X_train_left.shape[0] * 1. / BATCH_SIZE))
    #Làm tròn lên
    iter_losses = np.zeros(EPOCHS * iter_per_epoch)

    n_iter = 0

    perm_idx = np.random.permutation(X_train_left.shape[0])
    # Hoán vị ngẫu nhiên một dãy hoặc trả về một dãy đã hoán vị. - permutation

    # train for each batch

    for t_i in range(0, X_train_left.shape[0], BATCH_SIZE): # (0, 6526, 1)
        batch_idx = perm_idx[t_i:(t_i + BATCH_SIZE)]

        x_train_left_batch = np.take(X_train_left, batch_idx, axis=0)
        # Lấy ra 10 giá trị x_train_left ở vị trí batch_idx (1, 10, 6)

        x_train_right_batch = np.take(X_train_right, batch_idx, axis=0)
        # Lấy ra 10 giá trị x_train_right ở vị trí batch_idx (1, 10, 6)

        y_train_batch = np.take(y_train, batch_idx, axis=0)
        # Lấy ra 6 giá trị y_train cần điền khuyết ở vị trí batch_index (1,6,1)

        loss = train_iteration(model, optimizer, criterion, CLIP,x_train_left_batch, x_train_right_batch, y_train_batch)

        iter_losses[t_i // BATCH_SIZE] = loss

        n_iter += 1

    return np.mean(iter_losses[range(0, iter_per_epoch)])


def train_iteration(model, optimizer, criterion, clip, X_train_left, X_train_right, y_train):

    model.train()
    optimizer.zero_grad()

    X_train_left = np.transpose(X_train_left, [1, 0, 2])  # (10, 6526, 6)
    X_train_right = np.transpose(X_train_right, [1, 0, 2]) # (10, 6526, 6)
    #X_train_left22[9].shape (6526, 6)

    y_train = np.transpose(y_train, [1, 0, 2])  #(6, 6526, 1)

    X_train_left_tensor = numpy_to_tvar(X_train_left)  
    X_train_right_tensor = numpy_to_tvar(X_train_right)
    y_train_tensor = numpy_to_tvar(y_train)   # torch.Size([6, 6526, 1])

    output, atten = model(X_train_left_tensor, X_train_right_tensor, y_train_tensor)

    output = output.permute(1, 0, 2)
    y_train_tensor = y_train_tensor.permute(1, 0, 2)

    loss_mse, loss_shape, loss_temporal = torch.tensor(0), torch.tensor(0), torch.tensor(0)

    loss, loss_shape, loss_temporal = dilate_loss(y_train_tensor, output, 0.85, 0.01, device)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    optimizer.step()
    # # for AdamW+Cyclical Learning Rate

    return loss.item()



########## Đánh giá  ################
def evaluate(model, criterion, X_test_left, X_test_right, y_test):

  epoch_loss = 0
  iter_per_epoch = int(np.ceil(X_test_left.shape[0] * 1. / BATCH_SIZE))
  iter_losses = np.zeros(EPOCHS * iter_per_epoch)
  iter_multiloss = [np.zeros(EPOCHS * iter_per_epoch), np.zeros(EPOCHS * iter_per_epoch),
                    np.zeros(EPOCHS * iter_per_epoch), np.zeros(EPOCHS * iter_per_epoch)]
  perm_idx = np.random.permutation(X_test_left.shape[0])

  n_iter = 0

  with torch.no_grad():
    for t_i in range(0, X_test_left.shape[0], BATCH_SIZE):
      batch_idx = perm_idx[t_i:(t_i + BATCH_SIZE)]

      x_test_left_batch = np.take(X_test_left, batch_idx, axis=0)
      x_test_right_batch = np.take(X_test_right, batch_idx, axis=0)
      y_test_batch = np.take(y_test, batch_idx, axis=0)

      loss, mae, rmsle, rmse, loss_tdi = evaluate_iteration(model, criterion, x_test_left_batch, x_test_right_batch, y_test_batch)
      iter_losses[t_i // BATCH_SIZE] = loss
      iter_multiloss[0][t_i // BATCH_SIZE] = mae
      iter_multiloss[1][t_i // BATCH_SIZE] = rmsle
      iter_multiloss[2][t_i // BATCH_SIZE] = rmse
      iter_multiloss[3][t_i // BATCH_SIZE] = loss_tdi

      n_iter += 1

  return np.mean(iter_losses[range(0, iter_per_epoch)]), np.mean(iter_multiloss[0][range(0, iter_per_epoch)]), np.mean(
      iter_multiloss[1][range(0, iter_per_epoch)]), np.mean(iter_multiloss[2][range(0, iter_per_epoch)]), np.mean(iter_multiloss[3][range(0, iter_per_epoch)])


def evaluate_iteration(model, criterion, X_test_left, X_test_right, y_test):
  model.eval()

  x_test_left = np.transpose(X_test_left, [1, 0, 2])
  x_test_right = np.transpose(X_test_right, [1, 0, 2])
  y_test = np.transpose(y_test, [1, 0, 2])

  x_test_left_tensor = numpy_to_tvar(x_test_left)
  x_test_right_tensor = numpy_to_tvar(x_test_right)

  y_test_tensor = numpy_to_tvar(y_test)

  output, atten = model(x_test_left_tensor,x_test_right_tensor, y_test_tensor, 0)

  loss = criterion(output, y_test_tensor)
  loss_mse, loss_dtw, loss_tdi = 0, 0, 0
  loss_mae, loss_RMSLE, loss_RMSE = 0, 0, 0

  for k in range(BATCH_SIZE):
    target_k_cpu = y_test_tensor[:, k, 0:1].view(-1).detach().cpu().numpy()
    output_k_cpu = output[:, k, 0:1].view(-1).detach().cpu().numpy()

    loss_dtw += dtw(target_k_cpu, output_k_cpu)
    path, sim = dtw_path(target_k_cpu, output_k_cpu)

    Dist = 0
    for i, j in path:
        Dist += (i-j)*(i-j)
    loss_tdi += Dist / (N_output*N_output)

    loss_mae += mean_absolute_error(target_k_cpu, output_k_cpu)
    loss_RMSLE += np.sqrt(mean_squared_error(target_k_cpu, output_k_cpu))
    loss_RMSE += np.sqrt(mean_squared_error(target_k_cpu, output_k_cpu))

  loss_dtw = loss_dtw / BATCH_SIZE
  loss_tdi = loss_tdi / BATCH_SIZE
  loss_mae = loss_mae / BATCH_SIZE
  loss_RMSLE = loss_RMSLE / BATCH_SIZE
  loss_RMSE = loss_RMSE / BATCH_SIZE


  return loss.item(), loss_mae, loss_RMSLE, loss_RMSE, loss_dtw



########## Pridict ###########
def predict_ts(model, X_test_left, X_test_right, scaler_y, max_gap_size=6, BATCH_SIZE=1, device=device):
  model.eval()

  with torch.no_grad():

    x_test_left = np.transpose(X_test_left, [1, 0, 2])
    x_test_right = np.transpose(X_test_right, [1, 0, 2])

    empty_y_tensor = torch.zeros(max_gap_size, BATCH_SIZE,1).to(device)

    x_test_left_tensor = numpy_to_tvar(x_test_left)
    x_test_right_tensor = numpy_to_tvar(x_test_right)

    output, _ = model(x_test_left_tensor,x_test_right_tensor, empty_y_tensor, 0)

    output = torch.squeeze(output)
    output = torch.transpose(output, 0, 1)
    output = torch.flatten(output)

    # scalar
    output_numpy = output.cpu().data.numpy()
    output_numpy_origin = scaler_y.inverse_transform(output_numpy.reshape(-1, 1))

  return output_numpy_origin, output_numpy


In [7]:
### Chia train/test set

len_before = 10
len_after = 10
graphs = 20

## Train/test set cho NO3
(x_train, y_train), (x_test, y_test), (scaler_x,scaler_y) = test_qld_single_station(len_before, len_after, graphs)


## Train/test set cho Level
#(x_train, y_train), (x_test, y_test), (scaler_x,scaler_y) = test_qld_single_station_Level(len_before, len_after, graphs)

train_preprocess:(6551, 6)
test_preprocess:(2207, 6)
concatenated_x (6512, 40, 6)
concatenated_y (6512, 20)
len_all_case (6512,)
len_before_all_case (6512,)
x:(6512, 40, 6)
y:(6512, 20, 1)
concatenated_x (2168, 40, 6)
concatenated_y (2168, 20)
len_all_case (2168,)
len_before_all_case (2168,)
x:(2168, 40, 6)
y:(2168, 20, 1)
x_train:(6512, 40, 6)
y_train:(6512, 20, 1)
x_test:(2168, 40, 6)
y_test:(2168, 20, 1)
split train/test array
(2168, 10, 6)
(2168, 20, 6)
(2168, 10, 6)


In [8]:
# Dòng 250 - 251
print('split train/test array')
x_test_list = np.split(x_test, [len_before, len_before+graphs], axis=1)
x_train_list = np.split(x_train, [len_before, len_before+graphs], axis=1)

for i in x_test_list:
    print(i.shape)
print('==========')
for i in x_train_list:
    print(i.shape)

X_train_left = x_train_list[0]
X_train_right = x_train_list[2]
X_test_left = x_test_list[0]
X_test_right = x_test_list[2]

print('X_train_left:{}'.format(X_train_left.shape))
print('X_train_right:{}'.format(X_train_right.shape))
print('X_test_left:{}'.format(X_test_left.shape))
print('X_test_right:{}'.format(X_test_right.shape))

# fit batchsize, kiểm tra kích thước sau khi xóa khuyết
X_train_left = X_train_left[:]
X_train_right = X_train_right[:]
print(X_train_left.shape, X_train_right.shape)

split train/test array
(2168, 10, 6)
(2168, 20, 6)
(2168, 10, 6)
(6512, 10, 6)
(6512, 20, 6)
(6512, 10, 6)
X_train_left:(6512, 10, 6)
X_train_right:(6512, 10, 6)
X_test_left:(2168, 10, 6)
X_test_right:(2168, 10, 6)
(6512, 10, 6) (6512, 10, 6)


In [ ]:
'''pp='/content/stop_early_Level20.pt'
model.load_state_dict(torch.load(pp, map_location=torch.device('cpu')))

test_loss, test_mae, test_rmsle, test_rmse, test_tdi = evaluate(model, criterion, X_test_left, X_test_right, y_test)

print(f'| Test Loss: {test_loss:.4f} | Test PPL: {math.exp(test_loss):7.4f} |')
print(f'| MAE: {test_mae:.4f} | Test PPL: {math.exp(test_mae):7.4f} |')
print(f'| RMSLE: {test_rmsle:.4f} | Test PPL: {math.exp(test_rmsle):7.4f} |')
print(f'| RMSE: {test_rmse:.4f} | Test PPL: {math.exp(test_rmse):7.4f} |')
print(f'| DTW: {test_tdi:.4f} | Test PPL: {math.exp(test_tdi):7.4f} |')'''

"pp='/content/stop_early_Level20.pt'\nmodel.load_state_dict(torch.load(pp, map_location=torch.device('cpu')))\n\ntest_loss, test_mae, test_rmsle, test_rmse, test_tdi = evaluate(model, criterion, X_test_left, X_test_right, y_test)\n\nprint(f'| Test Loss: {test_loss:.4f} | Test PPL: {math.exp(test_loss):7.4f} |')\nprint(f'| MAE: {test_mae:.4f} | Test PPL: {math.exp(test_mae):7.4f} |')\nprint(f'| RMSLE: {test_rmsle:.4f} | Test PPL: {math.exp(test_rmsle):7.4f} |')\nprint(f'| RMSE: {test_rmse:.4f} | Test PPL: {math.exp(test_rmse):7.4f} |')\nprint(f'| DTW: {test_tdi:.4f} | Test PPL: {math.exp(test_tdi):7.4f} |')"

### Model

In [9]:
# Model
cross_attn = Cross_Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Shared_Encoder(INPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, ECN_Layers, DEC_Layers, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_Layers, DEC_DROPOUT, cross_attn)

model = DualSSIM(enc, dec, device).to(device)
model.apply(init_weights)

print(model)
print(f'The model has {count_parameters(model):,} trainable parameters')

DualSSIM(
  (shared_encoder): Shared_Encoder(
    (input_linear): Linear(in_features=6, out_features=50, bias=True)
    (gru_left): GRU(50, 50, bidirectional=True)
    (gru_right): GRU(50, 50, bidirectional=True)
    (output_linear_left): Linear(in_features=100, out_features=50, bias=True)
    (output_linear_right): Linear(in_features=100, out_features=50, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (attention): Cross_Attention(
      (attn): Linear(in_features=150, out_features=50, bias=True)
    )
    (input_dec): Linear(in_features=1, out_features=50, bias=True)
    (gru): GRU(150, 50)
    (out): Linear(in_features=200, out_features=1, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
The model has 109,851 trainable parameters


### Tối ưu hóa

In [10]:
#Adam
optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))

# warmup
scheduler_steplr = StepLR(optimizer, step_size=10, gamma=0.1)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=5)

criterion = nn.MSELoss()
# Dừng sớm
# Khởi tạo đối tượng dừng sớm
patience = 10

## Stop early cho NO3
#early_stopping = EarlyStopping(output_path='/content/stop_early.pt',patience=patience,verbose=True)


#Stop early cho Level
early_stopping = EarlyStopping(output_path='/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/Results_Kết quả chạy model_NO3/no3_20/stop_early_NO3_20.pt',patience=patience,verbose=True)

optimizer.zero_grad()
optimizer.step()

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.001
    lr: 0.0
    maximize: False
    weight_decay: 0
)

### Trainning

In [ ]:
best_valid_loss = float('inf')
for epoch in range(EPOCHS):

    scheduler_warmup.step(epoch)
    train_epoch_losses = np.zeros(EPOCHS)
    evaluate_epoch_losses = np.zeros(EPOCHS)


    start_time = time.time()
    train_loss = train(model, optimizer, criterion, X_train_left, X_train_right, y_train)
    valid_loss,test_mae, test_rmsle, test_rmse, test_tdi = evaluate(model, criterion, X_test_left, X_test_right, y_test)
    end_time = time.time()


    train_epoch_losses[epoch] = train_loss
    evaluate_epoch_losses[epoch] = valid_loss

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Dừng sớm cần đánh giá hàm loss để xem đã giảm hay chưa, và nếu có thì sẽ dừng ở mô hình hiện tại
    early_stopping(valid_loss, model)

    if early_stopping.early_stop:
         print("Early stopping")
         break
    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print( f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print( f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print(f'| MAE: {test_mae:.4f} | Test PPL: {math.exp(test_mae):7.4f} |')
    print(f'| RMSLE: {test_rmsle:.4f} | Test PPL: {math.exp(test_rmsle):7.4f} |')
    print(f'| RMSE: {test_rmse:.4f} | Test PPL: {math.exp(test_rmse):7.4f} |')
    print(f'| TDI: {test_tdi:.4f} | Test PPL: {math.exp(test_tdi):7.4f} |')

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/utils/dilate_loss.py:242: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  Omega =  pairwise_distances(torch.range(1,N_output).view(N_output,1)).to(device)
/content/gdrive/MyDrive/Project_HK1_2022/Data_Visu

Validation loss decreased (inf --> 0.148679).  Saving model ...
Epoch: 01 | Time: 5m 38s
	Train Loss: 0.217 | Train PPL:   1.242
	 Val. Loss: 0.149 |  Val. PPL:   1.160
| MAE: 0.2544 | Test PPL:  1.2897 |
| RMSLE: 0.2971 | Test PPL:  1.3460 |
| RMSE: 0.2971 | Test PPL:  1.3460 |
| TDI: 1.3287 | Test PPL:  3.7762 |


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/utils/dilate_loss.py:242: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  Omega =  pairwise_distances(torch.range(1,N_output).view(N_output,1)).to(device)


Validation loss decreased (0.148679 --> 0.094697).  Saving model ...
Epoch: 02 | Time: 5m 23s
	Train Loss: -0.148 | Train PPL:   0.863
	 Val. Loss: 0.095 |  Val. PPL:   1.099
| MAE: 0.2012 | Test PPL:  1.2229 |
| RMSLE: 0.2417 | Test PPL:  1.2734 |
| RMSE: 0.2417 | Test PPL:  1.2734 |
| TDI: 0.9633 | Test PPL:  2.6203 |


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/utils/dilate_loss.py:242: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  Omega =  pairwise_distances(torch.range(1,N_output).view(N_output,1)).to(device)


...

EarlyStopping counter: 8 out of 10
Epoch: 14 | Time: 5m 23s
	Train Loss: -0.176 | Train PPL:   0.838
	 Val. Loss: 0.065 |  Val. PPL:   1.067
| MAE: 0.1539 | Test PPL:  1.1663 |
| RMSLE: 0.1934 | Test PPL:  1.2133 |
| RMSE: 0.1934 | Test PPL:  1.2133 |
| TDI: 0.7176 | Test PPL:  2.0495 |


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/utils/dilate_loss.py:242: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  Omega =  pairwise_distances(torch.range(1,N_output).view(N_output,1)).to(device)


EarlyStopping counter: 9 out of 10
Epoch: 15 | Time: 5m 23s
	Train Loss: -0.176 | Train PPL:   0.839
	 Val. Loss: 0.075 |  Val. PPL:   1.078
| MAE: 0.1633 | Test PPL:  1.1774 |
| RMSLE: 0.2022 | Test PPL:  1.2240 |
| RMSE: 0.2022 | Test PPL:  1.2240 |
| TDI: 0.7139 | Test PPL:  2.0419 |


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/utils/dilate_loss.py:242: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  Omega =  pairwise_distances(torch.range(1,N_output).view(N_output,1)).to(device)


EarlyStopping counter: 10 out of 10
Early stopping


### Đánh giá

In [11]:
#Load NO3
#model.load_state_dict(torch.load('/content/stop_early.pt'))

#Load Level
pp='/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/Results_Kết quả chạy model_NO3/no3_20/stop_early_NO3_20.pt'

model.load_state_dict(torch.load(pp))

test_loss, test_mae, test_rmsle, test_rmse, test_tdi = evaluate(model, criterion, X_test_left, X_test_right, y_test)

print(f'| Test Loss: {test_loss:.4f} | Test PPL: {math.exp(test_loss):7.4f} |')
print(f'| MAE: {test_mae:.4f} | Test PPL: {math.exp(test_mae):7.4f} |')
print(f'| RMSLE: {test_rmsle:.4f} | Test PPL: {math.exp(test_rmsle):7.4f} |')
print(f'| RMSE: {test_rmse:.4f} | Test PPL: {math.exp(test_rmse):7.4f} |')
print(f'| DTW: {test_tdi:.4f} | Test PPL: {math.exp(test_tdi):7.4f} |')

| Test Loss: 0.0522 | Test PPL:  1.0536 |
| MAE: 0.1532 | Test PPL:  1.1656 |
| RMSLE: 0.1854 | Test PPL:  1.2037 |
| RMSE: 0.1854 | Test PPL:  1.2037 |
| DTW: 0.7019 | Test PPL:  2.0175 |


### Predict

In [ ]:
outputs_ori, outputs_scal = predict_ts(model, X_test_left, X_test_right, scaler_y, max_gap_size=6, BATCH_SIZE=1, device=device)

print('==================')
print('outputs_ori:{}'.format(outputs_ori.shape))
print('==================')
print('outputs_scal:{}'.format(outputs_scal.shape))


# N03
#np.save('/content/{}_ori'.format('Nitrate6_1012'), outputs_ori)
#np.save('/content/{}_scal'.format('Nitrate6_1012'), outputs_scal)

#/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/Results_Kết quả chạy model_NO3
#Level  
np.save('/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/Results_Kết quả chạy model_NO3/no3_20/{}_ori'.format('NO3_20_1012'), outputs_ori)
np.save('/content/gdrive/MyDrive/Project_HK1_2022/Data_Visualization/Code/Model_dienkhuyet/Results_Kết quả chạy model_NO3/no3_20/{}_scal'.format('NO3_20_1012'), outputs_scal)

outputs_ori:(13008, 1)
outputs_scal:(13008,)
